In [8]:
import pandas as pd
import numpy as np

# 1. Chargement des fichiers
stocks_df = pd.read_csv('../base/stocksV2.csv', sep=';')
exped_df = pd.read_csv('../base/expeditionsV2.csv', sep=';')

# 2. Aperçu des données
print("Stocks:\n", stocks_df.head(), "\n")
print("Expéditions:\n", exped_df.head(), "\n")

# 3. Infos et types
print(stocks_df.info())
print(exped_df.info())



Stocks:
          date id_produit entrepot  niveau_stock
0  01/06/2024     P00001     Lyon            88
1  22/04/2024     P00001    Paris           141
2  25/09/2024     P00002     Lyon           286
3  18/02/2024     P00003    Paris           177
4  13/07/2024     P00003     Lyon            95 

Expéditions:
   id_expedition id_commande id_produit  quantité   entrepôt date_expedition  \
0     SHP000001     O000001     P00413         2  Marseille      2024-09-09   
1     SHP000002     O000002     P00778        15       Lyon      2024-11-16   
2     SHP000003     O000004     P00535         8       Lyon      2024-11-06   
3     SHP000004     O000006     P00898         3      Paris      2024-01-26   
4     SHP000005     O000007     P00445         1       Lyon      2024-02-18   

  date_livraison transporteur  coût_transport  délai_livraison  
0     2024-09-15          UPS           11.62                6  
1     2024-11-23   Chronopost           13.61                7  
2     2024-11-09 

In [9]:

# 4. Valeurs manquantes
print("Valeurs manquantes dans stocks :\n", stocks_df.isnull().sum())
print("Valeurs manquantes dans expéditions :\n", exped_df.isnull().sum())


Valeurs manquantes dans stocks :
 date            0
id_produit      0
entrepot        0
niveau_stock    0
dtype: int64
Valeurs manquantes dans expéditions :
 id_expedition      0
id_commande        0
id_produit         0
quantité           0
entrepôt           0
date_expedition    0
date_livraison     0
transporteur       0
coût_transport     0
délai_livraison    0
dtype: int64


In [10]:

exped_df['date_expedition'] = pd.to_datetime(exped_df['date_expedition'], errors='coerce')
exped_df['date_livraison']=pd.to_datetime(exped_df['date_livraison'],errors='coerce')
stocks_df['date']=pd.to_datetime(stocks_df['date'],errors='coerce')
# 5. Doublons
stocks_df.drop_duplicates(inplace=True)
exped_df.drop_duplicates(inplace=True)


In [11]:
stocks_df.info(),exped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1706 entries, 0 to 1705
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          664 non-null    datetime64[ns]
 1   id_produit    1706 non-null   object        
 2   entrepot      1706 non-null   object        
 3   niveau_stock  1706 non-null   int64         
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 53.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42404 entries, 0 to 42403
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id_expedition    42404 non-null  object        
 1   id_commande      42404 non-null  object        
 2   id_produit       42404 non-null  object        
 3   quantité         42404 non-null  int64         
 4   entrepôt         42404 non-null  object        
 5   date_expedition  42404 non-null  dateti

(None, None)

In [ ]:

# 6. Valeurs aberrantes (z-score simple sur les colonnes numériques)
from scipy.stats import zscore

for col in ['niveau_stock', 'quantité', 'coût_transport','delai_livraison']:
    if col in stocks_df.columns:
        stocks_df = stocks_df[(np.abs(zscore(stocks_df[col])) < 3)]
    if col in exped_df.columns:
        exped_df = exped_df[(np.abs(zscore(exped_df[col])) < 3)]


In [13]:

# 8. Fusion possible si nécessaire par id_produit
merged_df = pd.merge(stocks_df, exped_df, on='id_produit', how='left')
